<a href="https://colab.research.google.com/github/pedroafleite/instacart-market-basket-analysis/blob/main/instacart_2_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.3.1/en_core_web_lg-2.3.1.tar.gz
#!pip install en_core_web_lg

In [ ]:
#pip install -U spacy

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import re, nltk, spacy, gensim
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/instacart/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/instacart/departments.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/instacart/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/instacart/order_products__train.csv')
orders = pd.read_csv('/content/drive/MyDrive/instacart/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/instacart/products.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/instacart/sample_submission.csv')

In [ ]:
order1 = order_products_prior.merge(products, how='inner')
order1 = order1.sort_values(by=['order_id','add_to_cart_order'])
order1.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
19400,2,28985,2,1,Michigan Organic Kale,83,4
86849,2,9327,3,0,Garlic Powder,104,13
93148,2,45918,4,1,Coconut Butter,19,13
93892,2,30035,5,0,Natural Sweetener,17,13


In [ ]:
order1.shape

(32434489, 7)

In [ ]:
# Yield successive n-sized 
# chunks from l. 
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n]

# How many elements each 
# list should have 
n = int(len(order1)/100)
  
x = list(divide_chunks(order1, n)) 

In [ ]:
order2 = x[0]
order2.shape

(324344, 7)

In [ ]:
df = order2.merge(orders, how='outer')
df.drop(["add_to_cart_order", "reordered", "eval_set", "order_number", 
         "order_dow", "order_hour_of_day", "days_since_prior_order"], axis=1, inplace=True)
df.head(10)

,order_id,product_id,product_name,aisle_id,department_id,user_id
0,2,33120.0,Organic Egg Whites,86.0,16.0,202279
1,2,28985.0,Michigan Organic Kale,83.0,4.0,202279
2,2,9327.0,Garlic Powder,104.0,13.0,202279
3,2,45918.0,Coconut Butter,19.0,13.0,202279
4,2,30035.0,Natural Sweetener,17.0,13.0,202279
5,2,17794.0,Carrots,83.0,4.0,202279
6,2,40141.0,Original Unflavored Gelatine Mix,105.0,13.0,202279
7,2,1819.0,All Natural No Stir Creamy Almond Butter,88.0,13.0,202279
8,2,43668.0,Classic Blend Cole Slaw,123.0,4.0,202279
9,3,33754.0,Total 2% with Strawberry Lowfat Greek Strained...,120.0,16.0,205970


In [ ]:
df.shape

(3713274, 6)

In [ ]:
df['user_id'].nunique()

206209

In [ ]:
df['order_id'].nunique() # more orders than users, as expected

3421083

In [ ]:
df.isna().sum()

order_id         0
product_id       0
product_name     0
aisle_id         0
department_id    0
user_id          0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('/content/drive/MyDrive/instacart/df2.csv',encoding='utf-8', index=False)

In [ ]:
def sent_to_words(sentences):
    ''' Tokenisation of sentences without punctuation'''
    
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True Remove the punctuation
        

data_words = list(sent_to_words(df["product_name"]))
print(data_words[:1])

[['organic', 'egg', 'whites']]


In [ ]:
#len(data_words)/5

In [ ]:
# Yield successive n-sized 
# chunks from l. 
#def divide_chunks(l, n): 
      
    # looping till length l 
#    for i in range(0, len(l), n):  
#        yield l[i:i + n]

# How many elements each 
# list should have 
#n = 276923
  
#x = list(divide_chunks(data_words, n)) 

In [ ]:
# 
#data_words = x[0]
len(data_words)

324344

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """It keeps the lemma of the words (lemma is the uninflected form of a word)"""
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' 
                                   for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
# You don't have permission in Colab to load the model with normal spacy usage  .
# Instead, import the model and load it directly:
#import en_core_web_lg
#nlp = en_core_web_lg.load()

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['organic egg white', '']


In [ ]:
# Create our products categories using LDA algorithm 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_features = 3000 
n_components = 5 # number of categories
n_top_words = 15


print("Extraction of tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(data_lemmatized)


def print_top_words(model, feature_names, n_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tf)

print("\n The categories in the LDA model :")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extraction of tf features for LDA...

 The categories in the LDA model :
Topic #0: organic water tomato strawberry butter bread sparkle sauce natural lemon grain wheat half raspberry fresh
Topic #1: original chocolate large cream almond egg milk vanilla bar chip unsweetened brown pure grade ice
Topic #2: organic banana bag apple juice green extra lime tortilla style oil flavor roll cucumber drink
Topic #3: organic cheese baby salt bean spinach cheddar potato sweet blueberry slice coconut carrot roast corn
Topic #4: organic milk free yogurt fat chicken gluten onion red white cracker grape breast snack reduce



In [ ]:
#Creation of the Categories variable giving the category of each product purchased
topics = lda.transform(tf)
df_topics = pd.DataFrame(topics)
list_topics = []

for index, row in df_topics.iterrows():
    column_line = str(row.idxmax(max(row)))
    list_topics.append(column_line)
    
df["Categories"] = pd.DataFrame(list_topics)

In [ ]:
df['Categories'].value_counts()

0    101156
3     63795
4     56413
2     52239
1     50741
Name: Categories, dtype: int64

In [ ]:
df.head()

,order_id,product_id,product_name,aisle_id,department_id,user_id,Categories
0,2,33120.0,Organic Egg Whites,86.0,16.0,202279,4
1,2,28985.0,Michigan Organic Kale,83.0,4.0,202279,0
2,2,9327.0,Garlic Powder,104.0,13.0,202279,0
3,2,45918.0,Coconut Butter,19.0,13.0,202279,0
4,2,30035.0,Natural Sweetener,17.0,13.0,202279,4


In [ ]:
df.dropna(axis=0, subset=["Categories"], inplace=True)

In [ ]:
# One hot encoding on the variable Categories

for category in df['Categories'].unique():
    df["Category {}".format(category)] = df['Categories'].str.contains(category).apply(lambda x:1 if x else 0)

In [ ]:
df.to_csv('/content/drive/MyDrive/instacart/df.csv',encoding='utf-8', index=False)